# Algorithm Selection

@roman

21 July, 2024

In [39]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import shap

from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, RegressorMixin
from scipy.stats import hmean

In [2]:
# Settings
# show 100 columns in pandas
pd.set_option('display.max_columns', 500)
TODAY = pd.to_datetime('today')

---
# Data

## Read

In [40]:
def get_properties_data(file_path, cols_to_stay, cols_as_categories):
    # Read database
    df = pd.read_parquet(file_path)

    # Set property_id as index
    df = df.set_index('property_id')

    # Handling NaNs
    df['elevador'] = df['elevador'].fillna(0)
    df['cve_vigilancia'] = df['cve_vigilancia'].fillna(0)
    df['tipo_vialidad'] = df['tipo_vialidad'].fillna(0)

    # Fill missing competitors values with terrain values
    df['competitors_weighted_mean_log_price_per_sqm'] = df['competitors_weighted_mean_log_price_per_sqm'].combine_first(df['mean_log_valor_fisico_terreno_m2'])
    df['competitors_weighted_mean_log_price_per_sqm_lower'] = df['competitors_weighted_mean_log_price_per_sqm_lower'].combine_first(df['mean_log_valor_fisico_terreno_m2_lower'])
    df['competitors_weighted_mean_log_price_per_sqm_upper'] = df['competitors_weighted_mean_log_price_per_sqm_upper'].combine_first(df['mean_log_valor_fisico_terreno_m2_upper'])

    # Casting integer columns
    columns_to_integer = ['cve_vigilancia', 'tipo_vialidad']
    df[columns_to_integer] = df[columns_to_integer].astype('float').round().astype('Int64')

    # Feature Engineering
    first_date_obs = df['fecha_avaluo'].min()
    last_date_obs = df['fecha_avaluo'].max()

    df = (
        df
        .assign(
            year_appraised=lambda x: x['fecha_avaluo'].dt.year,
            price_per_sqm=lambda x: x['valor_mercado'] / x['superficie_vendible'],
            quarters_since_first_appraisal=lambda x: (x['fecha_avaluo'] - first_date_obs).dt.days / (30.4 * 3),
            conservacion_recat=lambda x: x['conservacion'].replace({7: 3.5}) - x['conservacion'].min(),
            cve_vigilancia_recat=lambda x: np.where(x['cve_vigilancia'].eq(2), 1, 0),
            superficie_terreno_usable=lambda x: np.where(
                x['id_tipo_inmueble'].eq(4),
                x['superficie_accesoria'],
                x['superficie_terreno'] + x['superficie_accesoria']
            ),
            elevador=lambda x: x['elevador'].eq(1).astype('int'),
        )
    )

    # Cast columns as categories
    df[cols_as_categories] = df[cols_as_categories].astype('category')

    return df.loc[:, cols_to_stay]

# get data
cols_to_stay_with = [
    'id_clase_inmueble',
    'property_type',
    'elevador',
    'edad_anios',
    'year_appraised',
    'regimen_propiedad',
    'id_entidad_f',
    'banos',
    'medio_banos',
    'estacionamiento',
    'superficie_vendible',
    'superficie_terreno_usable',
    'distance_to_ocean',
    'longitud',
    'latitud',
    'count_supermarkets_at_1km',
    'count_hospitals_at_5km',
    'count_metro_at_1km',
    'count_schools_at_1km',
    'count_restaurants_at_1km',
    'competitors_weighted_mean_log_price_per_sqm',
    'mean_log_valor_fisico_terreno_m2',
    'mean_log_valor_fisico_terreno_m2_lower',
    'mean_log_valor_fisico_terreno_m2_upper',
    'quarters_since_first_appraisal',
    'conservacion_recat',
    'cve_vigilancia_recat',
    'price_per_sqm',
]

cols_to_categories = [
    'property_type', 'cve_vigilancia_recat', 'regimen_propiedad', 'id_entidad_f', 
]

df_properties = get_properties_data(
    "../../data/clean/properties_shif.parquet", cols_to_stay_with, cols_to_categories
    )

# see
print(df_properties.shape)
df_properties.head()

(852913, 28)


,id_clase_inmueble,property_type,elevador,edad_anios,year_appraised,regimen_propiedad,id_entidad_f,banos,medio_banos,estacionamiento,superficie_vendible,superficie_terreno_usable,distance_to_ocean,longitud,latitud,count_supermarkets_at_1km,count_hospitals_at_5km,count_metro_at_1km,count_schools_at_1km,count_restaurants_at_1km,competitors_weighted_mean_log_price_per_sqm,mean_log_valor_fisico_terreno_m2,mean_log_valor_fisico_terreno_m2_lower,mean_log_valor_fisico_terreno_m2_upper,quarters_since_first_appraisal,conservacion_recat,cve_vigilancia_recat,price_per_sqm
property_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66cf10199ef943a5a5ff82129e53d5d7,4,house,1,25,2020,PRIVADA INDIVIDUAL,09,1,0,0,348,143,inf,-99.158103,19.310875,2,12,0,0,7,9.719780,9.673058,8.712578,10.633539,5.614035,1.0,0,12371.479885
f84f9a0c784f491eab6bb100b513a95b,3,house,0,25,2020,PRIVADA INDIVIDUAL,15,2,0,1,108,113,inf,-99.065361,19.491722,0,4,0,0,0,9.565254,9.002990,8.205054,9.800925,6.239035,1.0,0,12515.574074
7ddd5a94feed4955a809f986d95722d8,4,house,0,1,2020,PRIVADA INDIVIDUAL,15,2,0,1,170,200,inf,-99.213541,19.633393,1,3,0,1,0,9.660476,8.285661,7.541603,9.029719,7.050439,1.0,0,14970.000000
d2e75fb3b385461995bf8d34b9d1fdfb,4,house,0,35,2020,PRIVADA INDIVIDUAL,09,2,0,0,139,107,inf,-99.114525,19.333061,2,7,1,1,0,9.946884,9.743626,9.015548,10.471704,7.236842,1.0,0,20906.474820
6f1194f9853443219e63f4a3222b010e,4,house,0,26,2020,PRIVADA INDIVIDUAL,15,1,0,0,54,84,inf,-99.052833,19.486586,2,2,2,0,0,9.445158,9.116628,8.610014,9.623243,5.614035,1.0,0,9920.314815


In [4]:
# count dtypes
df_properties.dtypes.value_counts()

int64       13
float64     10
category     1
category     1
category     1
category     1
Name: count, dtype: int64

In [5]:
# see nans
df_properties.isna().sum()[df_properties.isna().sum() > 0]

Series([], dtype: int64)

## Split

In [6]:
# split data (index)
index_train, index_test = train_test_split(
    df_properties.index, test_size=0.1, random_state=42, stratify=df_properties['property_type']
    )

# sizes
print(f"Train size: {len(index_train)}")
print(f"Test size: {len(index_test)}")

Train size: 767621
Test size: 85292


In [7]:
# count of property types
df_properties.loc[index_train, 'property_type'].value_counts(normalize=True)

property_type
house        0.756896
apartment    0.243104
Name: proportion, dtype: float64

In [8]:
# count of property types
df_properties.loc[index_test, 'property_type'].value_counts(normalize=True)

property_type
house        0.756894
apartment    0.243106
Name: proportion, dtype: float64

---
# Models

In [36]:
def calculate_metrics(y, y_pred, best_percent=1.0):
    # Create a DataFrame to hold y, y_pred, and MAPE
    df = pd.DataFrame({
        'y': y,
        'y_pred': y_pred
    })
    
    # Calculate MAPE
    df['mape'] = np.abs((df['y'] - df['y_pred']) / df['y'])
    
    # Determine the threshold MAPE to filter the best_percent data
    threshold_mape = df['mape'].quantile(best_percent)
    
    # Filter the best_percent of the data
    df_best = df[df['mape'] <= threshold_mape]
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(df_best['y'], df_best['y_pred']))
    mae = mean_absolute_error(df_best['y'], df_best['y_pred'])
    mape_best = df_best['mape'].mean()
    r2 = r2_score(df_best['y'], df_best['y_pred'])
    
    return {
        "rmse": rmse,
        "mape": mape_best,
        "mae": mae,
        "r2": r2
    }

## Baseline Model

## Fit

In [60]:
class BaselineModel(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.harmonic_means_ = {}
        self.default_mean_ = None
        self.first_category_ = None

    def fit(self, X, y):
        # Ensure X is a DataFrame and has exactly 2 columns
        if not isinstance(X, pd.DataFrame) or X.shape[1] != 2:
            raise ValueError("X must be a DataFrame with exactly 2 columns.")
        
        # Ensure y is a Series or ndarray with the same length as X
        if len(y) != len(X):
            raise ValueError("Length of y must be equal to the number of rows in X.")
        
        # Create a DataFrame with y included
        df = X.copy()
        df['y'] = y
        
        # Calculate harmonic means for each combination of categories
        grouped = df.groupby(list(X.columns))['y']
        self.harmonic_means_ = grouped.apply(lambda grp: hmean(grp)).to_dict()
        
        # Calculate the default mean as the harmonic mean of the first category
        self.first_category_ = X.columns[0]
        first_category = df.iloc[0][self.first_category_]
        
        if first_category:
            first_category_df = df[df[self.first_category_] == first_category]
            if not first_category_df.empty:
                # Calculate harmonic mean for each level of the second category
                category_means = first_category_df.groupby(X.columns[1])['y'].apply(lambda grp: hmean(grp))
                # Calculate the overall harmonic mean of these category means
                if not category_means.empty:
                    self.default_mean_ = hmean(category_means.values)
                else:
                    self.default_mean_ = np.nan
            else:
                self.default_mean_ = np.nan
        else:
            self.default_mean_ = np.nan
        
        return self

    def predict(self, X):
        # Ensure X is a DataFrame and has exactly 2 columns
        if not isinstance(X, pd.DataFrame) or X.shape[1] != 2:
            raise ValueError("X must be a DataFrame with exactly 2 columns.")
        
        # Create a DataFrame for predictions
        X_copy = X.copy()
        X_copy['prediction'] = X_copy.apply(lambda row: self.harmonic_means_.get(tuple(row), self.default_mean_), axis=1)
        return X_copy['prediction'].values


In [58]:
# cols to use
cols_x = [
    'id_entidad_f', 'year_appraised'
]

# x_train, y_train
X_train = df_properties.drop(columns=['price_per_sqm']).loc[index_train, cols_x].copy()
y_train = df_properties['price_per_sqm'].loc[index_train].copy()

# x_test, y_test
X_test = df_properties.drop(columns=['price_per_sqm']).loc[index_test, cols_x].copy()
y_test = df_properties['price_per_sqm'].loc[index_test].copy()

In [61]:
# create model
baseline_model = BaselineModel()

# fit
baseline_model.fit(X_train, y_train)

/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_70300/1481336516.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(list(X.columns))['y']


BaselineModel()

## Metrics

In [64]:
# train 
y_train_pred = baseline_model.predict(X_train)
calculate_metrics(y_train, y_train_pred)

{'rmse': 7085.723246096054,
 'mape': 0.2222027651888658,
 'mae': 3740.244497819799,
 'r2': 0.39301951518697664}

In [66]:
# test
y_test_pred = baseline_model.predict(X_test)
calculate_metrics(y_test, y_test_pred)

{'rmse': 7155.525326387041,
 'mape': 0.2223504754586003,
 'mae': 3753.829035604206,
 'r2': 0.3906267152340144}

---
# Sandbox